# I don't Know why I keep it, but it works ╮(￣▽￣")╭

In [1]:
def pip_install(*packages):
    '''
    Author: Sy Hwang, Hollis Nolan, Shuheng Liu 
    
    Install packages using pip
    Alternatively just use command line
    pip install package_name
    '''
    import pip
    if int(pip.__version__.split(".")[0]) >= 10:  # since 10.0.0, pip does not support pip.main() any more
        from pip._internal import main
    else:
        from pip import main

    for package in packages:
        try:
            main(["install", "--upgrade", package, "--user"])
        except Exception as e:
            print("Unable to install {} using pip.".format(package))
            print("Exception:", e)

In [2]:
pip_install('gensim', 'nltk')

Requirement already up-to-date: gensim in c:\users\pipi\appdata\roaming\python\python37\site-packages (3.8.0)
  Using cached https://files.pythonhosted.org/packages/87/16/4d247e27c55a7b6412e7c4c86f2500ae61afcbf5932b9e3491f8462f8d9e/nltk-3.4.4.zip


Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: 'C:\\Users\\pipi\\AppData\\Local\\Temp\\pip-req-tracker-2tfhw0mn\\b3095b1324fb85a822887988cbb36067fa60285c59804dcfe133f1c4'



In [3]:
#Find whether a Str is just a number
def is_number(s):
    try: 
        float(s)
        return True
    except ValueError:  
        pass 
    try:
        import unicodedata  
        unicodedata.numeric(s)  
        return True
    except (TypeError, ValueError):
        pass
    return False


#Find whether a Str contain English words or just meanless symbols
def containenglish(str0):
    import re
    return bool(re.search('[a-z]', str0))

# Clean all non-English reviews and meanless reviews

In [4]:
import pandas as pd
from langdetect import detect
import types

#Load movie reviews dataset
df = pd.read_csv( 'movie_reviews.csv', nrows=100000)
texts = df.text.values #pd.Series -> np.ndarray
label = df.label.values

#Create null lists 
Eng_list = []
label_list = []

#clean data
for i in range(len(label)): 
    if containenglish(texts[i]) == True:
        if (detect(texts[i]) == 'en'):
            Eng_list.append(texts[i])
            label_list.append(label[i])

In [5]:
import nltk
# Transform each review string as a list of token strings. May take a few seconds
tokenized = [nltk.word_tokenize(review) for review in Eng_list]

In [6]:
from nltk.corpus import stopwords
from string import punctuation
from itertools import chain

def clean_text(tokenized_list, sw, punct, lemmatize=False):
    new_list = []
    for doc in tokenized_list:
        new_list.append([token.lower() for token in doc if token.lower() not in chain(punct, sw)])
    return new_list

# Remove punctuations and stopwords, and lower-case text
sw = stopwords.words('english')
punct = punctuation
cleaned = clean_text(tokenized, sw, punct)


# Train the Doc2Vec Model

In [7]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import reuters

# Tokenize Reuters corpus
tokenized_docs = [nltk.word_tokenize(reuters.raw(fileid)) for fileid in reuters.fileids()]

# Convert tokenized documents to TaggedDocuments
tagged_docs = [TaggedDocument(doc, tags=[idx]) for idx, doc in enumerate(tokenized_docs)]

# Create and train the doc2vec model. May take a few seconds
doc2vec = Doc2Vec(vector_size=300, window=5, min_count=5, dm = 1, epochs=20)

# Build the word2vec model from the corpus
doc2vec.build_vocab(tagged_docs)

doc2vec.train(tagged_docs, epochs=20, total_examples=doc2vec.corpus_count)

'pattern' package not found; tag filters are not available for English
collecting all words and their counts
PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
PROGRESS: at example #10000, processed 1432016 words (7635877/s), 60879 word types, 10000 tags
collected 63346 word types and 10788 unique tags from a corpus of 10788 examples and 1548597 words
Loading a fresh vocabulary
effective_min_count=5 retains 15255 unique words (24% of original 63346, drops 48091)
effective_min_count=5 leaves 1472902 word corpus (95% of original 1548597, drops 75695)
deleting the raw counts dictionary of 63346 items
sample=0.001 downsamples 45 most-common words
downsampling leaves estimated 1118205 word corpus (75.9% of prior 1472902)
estimated required memory for 15255 words and 300 dimensions: 57185100 bytes
resetting layer weights
training model with 3 workers on 15255 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
EPOCH 1 - PROGRESS: at 98.20% exampl

# Vectorize the whole doc

In [8]:
Eng_vec = []
for i in cleaned:
    Eng_vec.append(doc2vec.infer_vector(i))

In [9]:
from sklearn.model_selection import train_test_split
import numpy as np

# split data into train and test sets
seed = 10
test_size = 0.4
X_train, X_test, y_train, y_test = train_test_split(Eng_vec, label_list, test_size=test_size, random_state=seed)

# Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

LRmodel=LogisticRegression(solver='liblinear')
LRmodel = LRmodel.fit(X_train, y_train)

y_pred = LRmodel.predict(X_test)


target_names = ['0', '1']

print(classification_report(y_test, y_pred, target_names=target_names))

# check the accuracy on the training set
LRmodel.score(X_test, y_test)

              precision    recall  f1-score   support

           0       0.55      0.17      0.26     14265
           1       0.65      0.92      0.76     24326

   micro avg       0.64      0.64      0.64     38591
   macro avg       0.60      0.54      0.51     38591
weighted avg       0.62      0.64      0.58     38591



0.6419631520302661

# Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

rdf = RandomForestClassifier(criterion='gini',n_estimators=10, random_state=0, max_depth=10, min_samples_split=10, bootstrap=True)
rdf.fit(X_train, y_train)

rdf_pred = rdf.predict(X_test)
target_names = ['0', '1']

print(classification_report(y_test, y_pred, target_names=target_names))

rdf_score = rdf.score(X_test, y_test)
print(rdf_score)

              precision    recall  f1-score   support

           0       0.55      0.17      0.26     14265
           1       0.65      0.92      0.76     24326

   micro avg       0.64      0.64      0.64     38591
   macro avg       0.60      0.54      0.51     38591
weighted avg       0.62      0.64      0.58     38591

0.6313907387732891


# Practices

In [12]:
n = 10 #arbitrary pick
print('Example review:\n   Raw: {} \n\n   Tokenized: {}'.format(texts[n], [i for i in tokenized[n]]))

Example review:
   Raw: So ingenious in concept, design and execution that you could watch it on a postage stamp-sized screen and still be engulfed by its charm. 

   Tokenized: ['Turns', 'out', 'the', 'real', 'magic', 'is', 'nothing', 'to', 'do', 'with', 'technology', ':', 'it', "'s", 'in', 'the', 'words', ',', 'the', 'voices', ',', 'the', 'story', '.', 'But', 'then', ',', 'a', 'child', 'could', 'have', 'told', 'you', 'that', '.']


In [13]:
from collections import Counter

#Note that we convert all tokens to lower case, otherwise words like *The* and *the* are different tokens.
token_counter = Counter(token.lower() for sentence in tokenized for token in sentence)
top10 = token_counter.most_common()[:10]
for i, t in enumerate(top10):
    print('{:>2}.{:>5}  freq: {:>7}'.format(i+1, t[0], t[1]))

 1.    .  freq:   99871
 2.    ,  freq:   94514
 3.  the  freq:   94348
 4.    a  freq:   65722
 5.   of  freq:   57975
 6.  and  freq:   57131
 7.   to  freq:   36159
 8.   's  freq:   34424
 9.   is  freq:   33839
10.   it  freq:   30807


In [14]:
top10 = cleaned[:10]
for i, t in enumerate(top10):
    print('{:>2}.{:>10}  freq: {:>10}   next: {:>10}'.format(i+1, t[0], t[1], t[2]))

 1.    entire  freq: generation   next:  filmgoers
 2.     pixar  freq:    classic   next:        one
 3.     woody  freq:      perks   next:    opening
 4.introduced  freq:        one   next:        two
 5.      easy  freq:        see   next:  virtually
 6.    though  freq:  animation   next:      seems
 7.   perhaps  freq:       film   next:      meant
 8.      time  freq:       kind   next:     future
 9.     think  freq:      speak   next:     adults
10. ingenious  freq:    concept   next:     design


In [15]:
from gensim import corpora

# Create a dictionary from list of documents in order to create BOW model
dictionary = corpora.Dictionary(cleaned)

adding document #0 to Dictionary(0 unique tokens: [])
adding document #10000 to Dictionary(18915 unique tokens: ['...', 'entire', 'ever', 'filmgoers', 'generation']...)
adding document #20000 to Dictionary(27122 unique tokens: ['...', 'entire', 'ever', 'filmgoers', 'generation']...)
adding document #30000 to Dictionary(32947 unique tokens: ['...', 'entire', 'ever', 'filmgoers', 'generation']...)
adding document #40000 to Dictionary(37856 unique tokens: ['...', 'entire', 'ever', 'filmgoers', 'generation']...)
adding document #50000 to Dictionary(42416 unique tokens: ['...', 'entire', 'ever', 'filmgoers', 'generation']...)
adding document #60000 to Dictionary(46736 unique tokens: ['...', 'entire', 'ever', 'filmgoers', 'generation']...)
adding document #70000 to Dictionary(50443 unique tokens: ['...', 'entire', 'ever', 'filmgoers', 'generation']...)
adding document #80000 to Dictionary(53999 unique tokens: ['...', 'entire', 'ever', 'filmgoers', 'generation']...)
adding document #90000 to 

In [16]:
corpus = [dictionary.doc2bow(text) for text in cleaned]
from gensim import corpora, models

#Create a TFIDF Model for the corpus
tfidf = models.TfidfModel(corpus)
print('Example review featurized with Corpus scores : \n{}'.format([(dictionary[i[0]], i[1]) for i in corpus[2]]))
print('Example review featurized with TF-IDF scores : \n{}'.format([(dictionary[i[0]], round(i[1],3)) for i in tfidf[corpus[2]]]))

collecting document frequencies
PROGRESS: processing document #0
PROGRESS: processing document #10000
PROGRESS: processing document #20000
PROGRESS: processing document #30000
PROGRESS: processing document #40000
PROGRESS: processing document #50000
PROGRESS: processing document #60000
PROGRESS: processing document #70000
PROGRESS: processing document #80000
PROGRESS: processing document #90000
calculating IDF weights for 96476 documents and 60169 features (1074117 matrix non-zeros)
Example review featurized with Corpus scores : 
[("'re", 1), ("'s", 1), ('alive', 1), ('art', 1), ('comes', 1), ('cowboy', 1), ('form', 1), ('opening', 1), ('perks', 1), ('rebirth', 1), ('scene', 1), ('toy', 1), ('watching', 1), ('woody', 1)]
Example review featurized with TF-IDF scores : 
[("'re", 0.179), ("'s", 0.051), ('alive', 0.262), ('art', 0.209), ('comes', 0.197), ('cowboy', 0.316), ('form', 0.231), ('opening', 0.256), ('perks', 0.448), ('rebirth', 0.377), ('scene', 0.214), ('toy', 0.326), ('watchin

In [17]:
from nltk.corpus import gutenberg
from gensim import models

# Training word2vec model on Gutenberg corpus. This may take a few minutes.
model = models.Word2Vec(cleaned,
                        size = 300,
                        window = 5,
                        min_count = 3,
                        sg = 0,
                        alpha = 0.025,
                        iter = 10,
                        batch_words = 10000)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 113993 words, keeping 18915 word types
PROGRESS: at sentence #20000, processed 229596 words, keeping 27122 word types
PROGRESS: at sentence #30000, processed 339919 words, keeping 32947 word types
PROGRESS: at sentence #40000, processed 451212 words, keeping 37856 word types
PROGRESS: at sentence #50000, processed 561821 words, keeping 42416 word types
PROGRESS: at sentence #60000, processed 676609 words, keeping 46736 word types
PROGRESS: at sentence #70000, processed 790298 words, keeping 50443 word types
PROGRESS: at sentence #80000, processed 904568 words, keeping 53999 word types
PROGRESS: at sentence #90000, processed 1026110 words, keeping 57742 word types
collected 60169 word types from a corpus of 1109695 raw words and 96476 sentences
Loading a fresh vocabulary
effective_min_count=3 retains 25721 unique words (42% of original 60169, dr

In [18]:
trained_text = model.train(cleaned, total_examples=100000, epochs=1)

Effective 'alpha' higher than previous training cycles


training model with 3 workers on 25721 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
worker thread finished; awaiting finish of 2 more threads
worker thread finished; awaiting finish of 1 more threads
worker thread finished; awaiting finish of 0 more threads
EPOCH - 1 : training on 1109695 raw words (993938 effective words) took 0.8s, 1322723 effective words/s


EPOCH - 1 : supplied example count (96476) did not equal expected count (100000)


training on a 1109695 raw words (993938 effective words) took 0.8s, 1313075 effective words/s


In [19]:
from sklearn.model_selection import train_test_split
import numpy as np

# split data into train and test sets
seed = 0
test_size = 0.33
word = []
#for i in cleaned:
    #for j in i:
word.append(model.wv[cleaned[2]])

#vector = model.wv['computer']  # numpy vector of a word
#X_train, X_test, y_train, y_test = train_test_split(model.wv.array, test_size=test_size, random_state=seed)

KeyError: "word 'perks' not in vocabulary"

In [ ]:
print('Example review featurized with TF-IDF scores : \n{}{}'.format(word[0], number[1]))

In [ ]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import reuters

from nltk.corpus import gutenberg
# Tokenize Reuters corpus
new_tokenized_docs = [nltk.word_tokenize(gutenberg.sent) ]

# Convert tokenized documents to TaggedDocuments
new_tagged_docs = [TaggedDocument(doc, tags=[idx]) for idx, doc in enumerate(tokenized_docs)]

# Create and train the doc2vec model. May take a few seconds
doc2vec = Doc2Vec(size=300, window=5, min_count=5, dm = 1, iter=10)

# Build the word2vec model from the corpus
doc2vec.build_vocab(tagged_docs)

In [ ]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import reuters
from nltk.corpus import gutenberg
from gensim import models
